In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np


INPUT_DIR = "./outputs/tables"
MASTER_FILE = "./outputs/master_monthly.csv"
OUTPUT_FIG_DIR = "./outputs/figures"
OUTPUT_TBL_DIR = "./outputs/tables"


os.makedirs(OUTPUT_FIG_DIR, exist_ok=True)
os.makedirs(OUTPUT_TBL_DIR, exist_ok=True)

# Data Loading and Preparation
Load the master dataset and the top 25 ranking tables. Ensure date columns are properly parsed for time-series analysis.

In [ ]:

try:
    df_master = pd.read_csv(MASTER_FILE)
    df_top_assi = pd.read_csv(os.path.join(INPUT_DIR, "top25_max_assi.csv"))
    df_top_upr = pd.read_csv(os.path.join(INPUT_DIR, "top25_max_upr.csv"))
    df_top_vol = pd.read_csv(os.path.join(INPUT_DIR, "top25_volatility.csv"))
    

    anom_path = os.path.join(INPUT_DIR, "anomalies_assi.csv")
    if os.path.exists(anom_path):
        df_anomalies = pd.read_csv(anom_path)
    else:
        df_anomalies = pd.DataFrame(columns=['month_year', 'state', 'district', 'pincode'])


    df_master['date'] = pd.to_datetime(df_master['month_year'], format='%Y-%m')
    df_anomalies['date'] = pd.to_datetime(df_anomalies['month_year'], format='%Y-%m', errors='coerce')
    
    print("Data loaded successfully.")
except Exception as e:
    print(f"Error loading data: {e}")
    df_master = pd.DataFrame()

Data loaded successfully.


# Case Selection Logic
Select 3 unique locations: one with highest peak ASSI, one with highest UPR, and one with highest volatility.

In [ ]:
selected_cases = []
seen_pincodes = set()


def get_top_unique(source_df, metric_col, seen_set):
    if source_df.empty: return None
    sorted_df = source_df.sort_values(metric_col, ascending=False)
    for _, row in sorted_df.iterrows():
        pid = (row['state'], row['district'], row['pincode'])
        pin = row['pincode']
        if pin not in seen_set:
            seen_set.add(pin)
            return row
    return None

# Case 1: Max ASSI
c1 = get_top_unique(df_top_assi, 'assi', seen_pincodes)
if c1 is not None: selected_cases.append({'type': 'Max Stress (ASSI)', 'data': c1})

# Case 2: Max UPR
c2 = get_top_unique(df_top_upr, 'upr', seen_pincodes)
if c2 is not None: selected_cases.append({'type': 'Max Update Pressure', 'data': c2})

# Case 3: Max Volatility
c3 = get_top_unique(df_top_vol, 'vol6', seen_pincodes)
if c3 is not None: selected_cases.append({'type': 'High Volatility', 'data': c3})

print(f"Selected {len(selected_cases)} cases for deep dive.")

Selected 3 cases for deep dive.


# Visualisation and Reporting
Loop through the selected cases, generate a 3-panel time-series plot for each, calculate summary statistics, and save the results.

In [ ]:
summary_rows = []

for i, case in enumerate(selected_cases):
    case_num = i + 1
    meta = case['data']
    case_type = case['type']
    
    state, district, pin = meta['state'], meta['district'], meta['pincode']
    
    # Filter Master Data
    mask = (df_master['state'] == state) & \
           (df_master['district'] == district) & \
           (df_master['pincode'] == pin)
    
    sub = df_master[mask].sort_values('date')
    
    if sub.empty:
        print(f"No master data for {pin}")
        continue

    # Filter Anomalies
    mask_anom = (df_anomalies['state'] == state) & \
                (df_anomalies['district'] == district) & \
                (df_anomalies['pincode'] == pin)
    sub_anom = df_anomalies[mask_anom]
    
    fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=True)
    
    # 1. ASSI Trend
    axes[0].plot(sub['date'], sub['assi'], color='purple', linewidth=2, label='ASSI')
    axes[0].set_title(f"Case {case_num}: {district} - {pin} ({case_type})\nAadhaar Service Stress Index (ASSI)")
    axes[0].set_ylabel('ASSI Score')
    axes[0].grid(True, alpha=0.3)
    axes[0].legend()
    
    # 2. Components
    axes[1].plot(sub['date'], sub['enrol_total'], label='Enrolment', color='blue', marker='o', markersize=4)
    axes[1].plot(sub['date'], sub['demo_total'], label='Demographic', color='green', marker='s', markersize=4)
    axes[1].plot(sub['date'], sub['bio_total'], label='Biometric', color='orange', marker='^', markersize=4)
    axes[1].set_title("Service Volumes Breakdown")
    axes[1].set_ylabel("Count")
    axes[1].grid(True, alpha=0.3)
    axes[1].legend()
    
    # 3. Growth & Anomalies
    axes[2].plot(sub['date'], sub['assi_mom'], color='gray', linestyle='--', label='ASSI MoM Growth')
    
    # Add anomalies
    if not sub_anom.empty:
        axes[2].scatter(sub_anom['date'], sub_anom['anom_score'] if 'anom_score' in sub_anom.columns else [0]*len(sub_anom), 
                        color='red', s=100, zorder=5, label='Anomaly Detected')
    
    axes[2].set_title("Month-over-Month Growth & Anomalies")
    axes[2].set_ylabel("Growth Rate")
    axes[2].axhline(0, color='black', linewidth=0.5)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.xlabel("Date")
    plt.tight_layout()
    

    out_fig = os.path.join(OUTPUT_FIG_DIR, f"case_study_{case_num}.png")
    plt.savefig(out_fig, dpi=150)
    plt.close()
    print(f"Saved plot: {out_fig}")
    

    summ = {
        'case_id': case_num,
        'state': state,
        'district': district,
        'pincode': pin,
        'selection_reason': case_type,
        'period_start': sub['date'].min().strftime('%Y-%m'),
        'period_end': sub['date'].max().strftime('%Y-%m'),
        'mean_assi': round(sub['assi'].mean(), 2),
        'max_assi': round(sub['assi'].max(), 2),
        'mean_upr': round(sub['upr'].mean(), 2) if 'upr' in sub else 0,
        'max_upr': round(sub['upr'].max(), 2) if 'upr' in sub else 0,
        'max_assi_mom': round(sub['assi_mom'].max(), 2) if 'assi_mom' in sub else 0,
        'volatility_latest': round(sub['vol6'].iloc[-1], 2) if 'vol6' in sub and len(sub) > 0 else 0,
        'anomaly_months_count': len(sub_anom),
        'key_insight_1': f"Peak load occurred in {sub.loc[sub['assi'].idxmax()]['month_year']}",
        'key_insight_2': f"Dominant workload is {'Enrolment' if sub['enrol_total'].sum() > sub['demo_total'].sum() else 'Updates'}",
        'uidai_action_recommendation': "Deploy mobile camp" if sub['enrol_total'].sum() > sub['demo_total'].sum() else "Set up permanent update desk"
    }
    summary_rows.append(summ)


df_summ = pd.DataFrame(summary_rows)
out_csv = os.path.join(OUTPUT_TBL_DIR, "case_study_summaries.csv")
df_summ.to_csv(out_csv, index=False)
print(f"Saved summary table: {out_csv}")
df_summ.head()

Saved plot: ./outputs/figures/case_study_1.png
Saved plot: ./outputs/figures/case_study_2.png
Saved plot: ./outputs/figures/case_study_3.png
Saved summary table: ./outputs/tables/case_study_summaries.csv


,case_id,state,district,pincode,selection_reason,period_start,period_end,mean_assi,max_assi,mean_upr,max_upr,max_assi_mom,volatility_latest,anomaly_months_count,key_insight_1,key_insight_2,uidai_action_recommendation
0,1,Uttar Pradesh,Moradabad,244001,Max Stress (ASSI),2025-03,2025-12,2111.53,3375.4,777.02,6962.0,1.07,0.37,0,Peak load occurred in 2025-07,Dominant workload is Enrolment,Deploy mobile camp
1,2,Delhi,North East Delhi,110094,Max Update Pressure,2025-03,2025-12,1698.83,3357.5,2726.73,12301.0,2.43,0.61,0,Peak load occurred in 2025-07,Dominant workload is Updates,Set up permanent update desk
2,3,Uttar Pradesh,Prayagraj,221505,High Volatility,2025-04,2025-12,34.61,248.0,6.04,17.0,82.00,2.16,1,Peak load occurred in 2025-07,Dominant workload is Enrolment,Deploy mobile camp
